[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/casangi/graphviper/blob/main/docs/graph_building_tutorial_image.ipynb)

# astroVIPER Tutorial: Image

In [1]:
import os

from importlib.metadata import version

try:
    import graphviper

    print("GraphVIPER version", version("graphviper"), "already installed.")
except ImportError as e:
    print(e)
    print("Installing GraphVIPER")

    os.system("pip install graphviper")

    import xradio

    print("GraphVIPER version", version("graphviper"), " installed.")

GraphVIPER version 0.0.1 already installed.


## Download dataset

In [2]:
from xradio.data.datasets import download

imname = "demo_simulated.im"
download(imname)

File exists.


## Setup Dask Cluster
To simplify things we are going to start of by just using a single thread (everything will run in serial).

In [3]:
import dask

dask.config.set(scheduler="synchronous")

## Inspect Image Dataset

In [4]:
from xradio.image import load_image, read_image, write_image

img_xds = read_image(infile="demo_simulated.im", chunks={"l": 40, "m": 20, "freq": 5})
img_xds

Successful readonly open of default-locked table demo_simulated.im: 1 columns, 1 rows


<xarray.Dataset>
Dimensions:          (time: 1, polarization: 4, frequency: 50, l: 200, m: 100,
                      beam_param: 3)
Coordinates:
  * time             (time) float64 5.154e+04
  * polarization     (polarization) <U1 'I' 'Q' 'U' 'V'
  * frequency        (frequency) float64 1.415e+09 1.415e+09 ... 1.415e+09
    velocity         (frequency) float64 1.146e+06 1.146e+06 ... 1.136e+06
  * l                (l) float64 0.02909 0.0288 0.02851 ... -0.02851 -0.0288
  * m                (m) float64 -0.01454 -0.01425 -0.01396 ... 0.01396 0.01425
    right_ascension  (l, m) float64 0.0291 0.0291 0.0291 ... 6.254 6.254 6.254
    declination      (l, m) float64 -0.01454 -0.01425 ... 0.01396 0.01425
  * beam_param       (beam_param) <U5 'major' 'minor' 'pa'
Data variables:
    sky              (time, polarization, frequency, l, m) float32 dask.array<chunksize=(1, 4, 50, 40, 20), meta=np.ndarray>
    mask0            (time, polarization, frequency, l, m) bool dask.array<chunksize=(1, 4, 50, 40, 20), meta=np.ndarray>
    beam             (time, polarization, frequency, beam_param) float64 dask.array<chunksize=(1, 4, 50, 3), meta=np.ndarray>
Attributes:
    direction:        {'reference': {'frame': 'FK5', 'type': 'sky_coord', 'eq...
    telescope:        {'name': 'ALMA', 'position': {'type': 'position', 'elli...
    obsdate:          {'type': 'time', 'scale': 'UTC', 'units': 'd', 'value':...
    pointing_center:  {'value': array([0., 0.]), 'initial': True}
    observer:         Karl Jansky
    object_name:      
    user:             {}
    active_mask:      mask0
    description:      None
    history:          <xarray.Dataset>\nDimensions:    (row: 15, TIME: 15, PR...

## Create Parallel Coordinates, Map

In [8]:
import numpy as np
import xarray as xr
import dask
from graphviper.graph_tools.coordinate_utils import make_parallel_coord
from graphviper.graph_tools.map import map
from graphviper.utils.display import dict_to_html
from IPython.display import HTML, display

from xradio.image import load_image, read_image, write_image


input_parms = {}
parallel_coords = {}
parallel_coords["frequency"] = make_parallel_coord(coord=img_xds.frequency, n_chunks=6)
sel_parms = {}
input_parms["input_data_store"] = "demo_simulated.im"

input_data = {"img": img_xds}

from graphviper.graph_tools.coordinate_utils import (
    interpolate_data_coords_onto_parallel_coords,
)

node_task_data_mapping = interpolate_data_coords_onto_parallel_coords(
    parallel_coords, input_data
)
display(HTML(dict_to_html(node_task_data_mapping)))


def my_func(input_parms):
    if input_parms["input_data"] is None:
        img_xds = load_image(
            input_parms["input_data_store"],
            block_des=input_parms["data_selection"]["img"],
        )
    else:
        img_xds = input_parms["input_data"]["img"]

    display(HTML(dict_to_html(input_parms)))
    print("****")


graph = map(
    input_data=input_data,
    node_task_data_mapping=node_task_data_mapping,
    node_task=my_func,
    input_parms=input_parms,
    in_memory_compute=True,
)
dask.compute(graph)

****


****


****


****


****


****


(([None, None, None, None, None, None], None),)